In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
# pd.options.plotting.backend = "plotly"

In [2]:
import scipy.io
mat =scipy.io.loadmat('/home/izmaylov/test/feature_selection/scikit-Dataset/lymphoma.mat')
X=mat['X']
y = mat['Y'][:, 0] 



In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [4]:
from skfeature.function.information_theoretical_based import MRMR
# num_fea = 5  
# idx  = MRMR.mrmr(X_train, y_train)
# selected_features_train = X_train[:, idx[0:num_fea]]
# selected_features_test = X_test[:, idx[0:num_fea]]

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

In [6]:
from sklearn.model_selection import GridSearchCV
#import logistic_regression 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#import naive_bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFdr

from tempfile import mkdtemp
from joblib import Memory
from shutil import rmtree

# search_space = [{'selector__k': [5, 10, 20, 30]},
#                 {'classifier': [LogisticRegression(solver='lbfgs')]},
#                 {'classifier': [RandomForestClassifier(n_estimators=100)]}]
#                 # {'classifier': [KNeighborsClassifier()]},
#                 # {'classifier': [GaussianNB()]},
#                 # {'classifier': [SVC()]}]
# # grid_search = GridSearchCV(pipe, param_grid=search_space)
cachedir = mkdtemp()
location = "cachedir"
memory = Memory(location=location, verbose=False)

# pipe = Pipeline([('selector', SelectKBest(mutual_info_classif, k=5)),
#                  ('classifier', LogisticRegression())],memory=memory)

In [7]:
import itertools
def make_param_grids(steps, param_grids):

    final_params=[]

    # Itertools.product will do a permutation such that 
    # (pca OR svd) AND (svm OR rf) will become ->
    # (pca, svm) , (pca, rf) , (svd, svm) , (svd, rf)
    for estimator_names in itertools.product(*steps.values()):
        current_grid = {}

        # Step_name and estimator_name should correspond
        # i.e preprocessor must be from pca and select.
        for step_name, estimator_name in zip(steps.keys(), estimator_names):
            for param, value in param_grids.get(estimator_name).items():
                if param == 'object':
                    # Set actual estimator in pipeline
                    current_grid[step_name]=[value]
                else:
                    # Set parameters corresponding to above estimator
                    current_grid[step_name+'__'+param]=value
        #Append this dictionary to final params            
        final_params.append(current_grid)

    return final_params

In [57]:
from sklearn.feature_selection import SelectFdr, chi2, f_classif
SelectFdr(f_classif, alpha=0.1)

"""example of param_grid:
https://stackoverflow.com/questions/42266737/parallel-pipeline-to-get-best-model-using-gridsearch

pipeline_steps = {'preprocessor':['pca', 'select'],
                  'classifier':['svm', 'rf']}
# fill parameters to be searched in this dict
all_param_grids = {'svm':{'object':SVC(),
                         }, 

                   'rf':{'object':RandomForestClassifier(),
                        },

                   'pca':{'object':PCA(),
                          'n_components':[10,20]
                         },

                   'select':{'object':SelectKBest(),
                             'k':[5,10]
                            }
                  }  

{
      "SelectFdr":{"object":{SelectFdr(f_classif, alpha=0.1)}},
      # "MRMR":{"object":{MRMR.mrmr()}},
      'pca':{'object':PCA()},
      # "LogisticRegression": {"object": LogisticRegression(solver='lbfgs')},
      # "RandomForest": {"object": RandomForestClassifier(n_estimators=100)},
      # "KNeighbors": {"object": KNeighborsClassifier()},
      "naive_bayes": {"object": GaussianNB()},
      # "SVD":      {"object": SVC()}
}
"""
from sklearn.base import BaseEstimator, TransformerMixin 
# import  sklearn.metrics._scorer:
from sklearn.metrics import make_scorer
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score

# class Test(BaseEstimator, TransformerMixin ):
#       def __init__(self, k=5):
#             self.k = k

#       def fit(self, X, y=None):
#             self.idx  = MRMR.mrmr(X_train, y_train)
#             return self

#       def fit_transform(self, X, y=None, **fit_params):
#         if y is None:
#             # fit method of arity 1 (unsupervised transformation)
#             return self.fit(X, **fit_params).transform(X)
#         else:
#             # fit method of arity 2 (supervised transformation)
#             return self.fit(X, y, **fit_params).transform(X)
            
#       def transform(self, X):
#             return X[:, self.idx[0:self.k]]

# def Test(*args):
#     print(args)
#     return args
# pipeline_steps = {'preprocessor':['pca', 'SelectFdr',"Mrmr"],
pipeline_steps = {'preprocessor':["MRMR","SelectFdr"],
                  'classifier':['naive_bayes',"KNeighbors"]}
# X_new = SelectKBest(score_func= MRMR.mrmr, k=20).fit_transform(X, y)
def on_step(optim_result):
      # print(optim_result)
      return True
#     score = searchcv.best_score_
#     print("best score: %s" % score)
#     if score >= 0.98:
#         print('Interrupting!')
#         return True

all_param_grids =   {
                  "SelectFdr":{"object":SelectFdr(),
                              "score_func":[f_classif],
                              "alpha":[0.1]},
                  'pca':{'object':PCA()},
                  # 'MRMR':{'object':SelectKBest(MRMR.mrmr),
                  'MRMR':{'object':SelectKBest(),
                             'k':[5],
                            },
                  "naive_bayes": {"object": GaussianNB()},
                  "KNeighbors": {"object": KNeighborsClassifier()},

            }
# t=Test()
# t.fit(X_train, y_train)
# t.transform(X_train)
# Call the method on the above declared variables
param_grids_list = make_param_grids(pipeline_steps, all_param_grids)
cachedir = mkdtemp()
location = "cachedir"
memory = Memory(location=location, verbose=False)

pipe = Pipeline(steps=[('preprocessor',PCA()), ('classifier', SVC())],memory=memory)  
scoring = {
      # "roc_auc": make_scorer(roc_auc_score),
            "accuracy": make_scorer(accuracy_score),
            # "matthews_corrcoef": make_scorer(matthews_corrcoef),
            # "PR-AUC": make_scorer(average_precision_score),
            }

print(param_grids_list)


grd = GridSearchCV(pipe, param_grid = param_grids_list,verbose=False, scoring=scoring,refit=False)
grd.fit(X, y,callback=[on_step])
print("f")

[{'preprocessor': [SelectKBest()], 'preprocessor__k': [5], 'classifier': [GaussianNB()]}, {'preprocessor': [SelectKBest()], 'preprocessor__k': [5], 'classifier': [KNeighborsClassifier()]}, {'preprocessor': [SelectFdr()], 'preprocessor__score_func': [<function f_classif at 0x7f0959f92cb0>], 'preprocessor__alpha': [0.1], 'classifier': [GaussianNB()]}, {'preprocessor': [SelectFdr()], 'preprocessor__score_func': [<function f_classif at 0x7f0959f92cb0>], 'preprocessor__alpha': [0.1], 'classifier': [KNeighborsClassifier()]}]
f


/home/izmaylov/.conda/envs/tf_cpu/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,
/home/izmaylov/.conda/envs/tf_cpu/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/izmaylov/.conda/envs/tf_cpu/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/izmaylov/.conda/envs/tf_cpu/lib/python3.7/site-

In [ ]:
estimator

In [38]:
memory.eval("grd.best_estimator_")

TypeError: 'str' object is not callable

In [26]:
#gridseach result to dataframe
df = pd.DataFrame(grd.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_preprocessor,param_preprocessor__k,param_preprocessor__alpha,param_preprocessor__score_func,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,0.015612,0.001883,0.002612,0.001253,GaussianNB(),SelectKBest(),5,NaN,NaN,"{'classifier': GaussianNB(), 'preprocessor': S...",0.30,0.684211,0.736842,0.578947,0.526316,0.565263,0.152090,3
1,0.012613,0.000110,0.003561,0.000128,KNeighborsClassifier(),SelectKBest(),5,NaN,NaN,"{'classifier': KNeighborsClassifier(), 'prepro...",0.45,0.631579,0.684211,0.578947,0.526316,0.574211,0.081407,2
2,0.020750,0.002364,0.012928,0.000590,GaussianNB(),SelectFdr(),NaN,0.1,<function f_classif at 0x7f0959f92cb0>,"{'classifier': GaussianNB(), 'preprocessor': S...",0.50,0.473684,0.473684,0.473684,0.473684,0.478947,0.010526,4
3,0.013567,0.001528,0.020003,0.025467,KNeighborsClassifier(),SelectFdr(),NaN,0.1,<function f_classif at 0x7f0959f92cb0>,"{'classifier': KNeighborsClassifier(), 'prepro...",0.90,0.894737,0.947368,0.894737,0.789474,0.885263,0.051825,1


In [34]:
grd.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [55]:
grd.estimator.named_steps['preprocessor'].get_params().keys()

dict_keys(['copy', 'iterated_power', 'n_components', 'random_state', 'svd_solver', 'tol', 'whiten'])

In [ ]:
fs = grd.best_estimator_.named_steps['fs']


In [10]:
pipe[:-1].get_feature_names_out()


AttributeError: Estimator preprocessor does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

In [49]:
x.classifier

AttributeError: 'GridSearchCV' object has no attribute 'classifier'

In [46]:
pipe.select

AttributeError: 'Pipeline' object has no attribute 'select'

In [38]:
pipe.steps[0]

('preprocessor', PCA())

In [35]:
pipe.get_params()["preprocessor"]clf.steps[0]

PCA()

In [30]:
X

array([[ 2,  2,  2, ...,  2,  0,  0],
       [ 0,  2,  2, ...,  0,  0, -2],
       [ 0, -2, -2, ...,  2,  2,  0],
       ...,
       [ 0, -2,  2, ...,  2,  0, -2],
       [ 2,  2,  2, ...,  2, -2, -2],
       [ 2,  2,  2, ...,  2,  2,  2]], dtype=int16)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_preprocessor,param_preprocessor__k,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,0.018851,0.003117,0.002743,0.000627,GaussianNB(),SelectKBest(),5,"{'classifier': GaussianNB(), 'preprocessor': S...",0.30,0.684211,0.736842,0.578947,0.526316,0.565263,0.152090,2
1,0.012831,0.000297,0.003987,0.000898,KNeighborsClassifier(),SelectKBest(),5,"{'classifier': KNeighborsClassifier(), 'prepro...",0.45,0.631579,0.684211,0.578947,0.526316,0.574211,0.081407,1


In [ ]:
memory.clear(warn=False)
rmtree(location)